In [22]:
from sklearn.model_selection import StratifiedKFold
from lazyqsar.qsar import LazyBinaryQSAR
import pandas as pd
import numpy as np
import lazyqsar
import tqdm
import os

In [23]:
root = "."

data = pd.read_csv(os.path.join(root, "..", "output", "mtuberculosis", "datasets", "CHEMBL4649948_IC50_umol.L-1_5.csv.gz"))

In [24]:
data

,compound_chembl_id,canonical_smiles,activity_type,value,relation,unit,bin
0,CHEMBL3455348,CC(Nc1ccc2c(c1)CCC2)c1nc(C2CC2)no1,IC50,0.18299,=,umol.L-1,1
1,CHEMBL3437665,CCn1nnc(NCc2ccccc2OCc2ccc(Cl)cc2)n1,IC50,0.30763,=,umol.L-1,1
2,CHEMBL3457035,CCOCC(=O)NC(C)Cc1ccccc1Br,IC50,0.74249,=,umol.L-1,1
3,CHEMBL3459598,Cc1nc(CN2CCC(CNC(=O)NCCC3CCC3)CC2)oc1C,IC50,0.79116,=,umol.L-1,1
4,CHEMBL3448847,O=C(CCC1CCCC1)N1CCC(c2cc3ncccc3c(O)n2)CC1,IC50,0.85460,=,umol.L-1,1
...,...,...,...,...,...,...,...
2461,CHEMBL3483073,O=C(c1ncoc1C1CC1)N1CCCN(CCCc2ccccc2)CC1,IC50,502.18723,=,umol.L-1,0
2462,CHEMBL4547998,CCNc1ncc(C(=O)N(C)Cc2ccsc2)cn1,IC50,502.18723,=,umol.L-1,0
2463,CHEMBL4521883,COc1ccc2c(c1)OCC(CNC(=O)CN(C)C)C2,IC50,502.18723,=,umol.L-1,0
2464,CHEMBL4583822,Cc1cc(NC(=O)Cc2ccc3c(c2)OCCO3)no1,IC50,502.18723,=,umol.L-1,0


In [25]:
X = data['canonical_smiles'].tolist()
Y = data['bin'].tolist()

X = np.array(X, dtype='str')
Y = np.array(Y, dtype=np.int8)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [26]:
for train, test in skf.split(X, Y):
    print(len(train), len(test))
    # clf = RandomForestClassifier(n_estimators=100, n_jobs=8, random_state=42)
    # print("Fitting model")
    # clf.fit(X[train], y[train])
    X_train, Y_train = X[train], Y[train]
    X_test, Y_test = X[test], Y[test]

    break

1972 494


In [29]:
model = LazyBinaryQSAR(mode="fast")
model.fit(smiles_list=X_train, y=Y_train)

18:53:10 INFO     Fitting with descriptor: morgan

Transforming Morgan descriptors in chunks of 1000: 100%|██████████| 1/1 [00:00<00:00,  2.59it/s]


         INFO     Dataset summary:                                                                                 
                    • Total samples: 1972                                                                          
                    • Positive samples: 98                                                                         
                    • Negative samples: 1874                                                                       
                    • Max samples per partition: 10000                                                             
                    • Min samples per partition: 30                                                                
                    • Positive proportion: 0.05

         INFO     Original positive samples: 98, total samples: 1972

         INFO     Maximum samples: 10000

...

19:01:38 INFO     Fitting completed

         INFO     Individual model scores:

         INFO     Model weights: [0.10088401 0.09097171 0.10396215 0.1082531  0.10785421 0.11027522                
                   0.10707488 0.08438657 0.08720186 0.09913628]

         INFO     Model has successfull been fitted!

         INFO     Fitting completed in 74.86 seconds.

In [30]:
model